In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
#url='http://pokemondb.net/pokedex/all'
#url = 'https://pokemondb.net/pokedex/game/sword-shield'
url = 'https://www.eurogamer.net/articles/2019-10-31-pokemon-sword-shield-galar-pokedex-6017'
page = requests.get(url)
doc = lh.fromstring(page.content)
#span_elements = doc.xpath('//span')
tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:12]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [4]:
col = []
col_names = ["Pokedex #", "Pokemon", "Type", "", "Generation"]
for t in col_names:
    name= t
#    print('%d: "%s"'%(i,name))
    col.append((name,[]))
col

[('Pokedex #', []),
 ('Pokemon', []),
 ('Type', []),
 ('', []),
 ('Generation', [])]

In [5]:
for j in range(0, len(tr_elements)):
#for j in range(1, len(span_elements)):
    T = tr_elements[j]
#    T = span_elements[j]
    if len(T) != 5:
        pass
    
    i = 0
    
    for t in T.iterchildren():
        data=t.text_content()
        if i > 0:
            try:
                data = int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [6]:
[len(C) for (title,C) in col]

[407, 407, 407, 407, 407]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
df

In [8]:
df = df[df["Pokedex #"] != 'TBC']

In [ ]:
df['Pokedex #']

In [9]:
for i in range(len(df['Pokedex #'])):
    x = df['Pokedex #'].iloc[i]
    x = x.split('#')
    if len(x) > 0:
        df['Pokedex #'].iloc[i] = x[1]

In [10]:
df.set_index('Pokedex #')

,Pokemon,Type,,Generation
Pokedex #,,,,
1,Grookey,Grass,,Gen 8 (SWSH)
2,Thwackey,Grass,,Gen 8 (SWSH)
3,Rillaboom,Grass,,Gen 8 (SWSH)
4,Scorbunny,Fire,,Gen 8 (SWSH)
5,Raboot,Fire,,Gen 8 (SWSH)
...,...,...,...,...
396,Drakloak,Dragon / Ghost,,Gen 8 (SWSH)
397,Dragapult,Dragon / Ghost,,Gen 8 (SWSH)
398,Zacian,Fairy (Fairy / Steel),,Gen 8 (SWSH)


In [13]:
url='https://www.dexerto.com/pokemon/pokemon-sword-and-shield-spawn-locations-guide-1287933'

page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [14]:
[len(T) for T in tr_elements[247:250]]

[4, 3, 4]

In [15]:
del tr_elements[248]


In [16]:
col = []
i = 0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d: "%s"'%(i,name))
    col.append((name,[]))

1: "Pokedex #"
2: "Pokemon"
3: "Overworld"
4: "Non-Overworld"


In [17]:
for j in range(1, len(tr_elements)):
    T = tr_elements[j]
    if len(T) != 4:
        break    
    i = 0
    
    for t in T.iterchildren():
        data=t.text_content()
        if i > 0:
            try:
                data = int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [26]:
Dict={title:column for (title,column) in col}
df_location=pd.DataFrame(Dict)

In [27]:
df_location.head()

,Pokedex #,Pokemon,Overworld,Non-Overworld
0,1,Grookey,Starter,
1,2,Thwackey,Evolve Grookey,
2,3,Rillaboom,Evolve Thwacket,
3,4,Scorbunny,Starter,
4,5,Raboot,Evolve Scorbunny,


In [28]:
d = {'Pokedex #': [248], 'Pokemon': ['Throh'], 'Overworld': ["Motostoke Outskirts, Bridge Field, Giant's Mirror"], 'Non-Overworld': [""]}
throh_df = pd.DataFrame(data=d)
throh_df

,Pokedex #,Pokemon,Overworld,Non-Overworld
0,248,Throh,"Motostoke Outskirts, Bridge Field, Giant's Mirror",


In [29]:
location_df = df_location.append(throh_df)

In [30]:
location_df['Pokedex #'] = location_df['Pokedex #'].astype(int)
location_df = location_df.set_index('Pokedex #')
location_df.sort_index()

In [35]:
pokemon_df = df.merge(location_df, on='Pokedex #')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [38]:
pokemon_df = pd.concat([df, location_df], axis=1)

In [39]:
pokemon_df

,Pokedex #,Pokemon,Type,,Generation,Pokemon,Overworld,Non-Overworld
0,1,Grookey,Grass,,Gen 8 (SWSH),NaN,NaN,NaN
1,2,Thwackey,Grass,,Gen 8 (SWSH),Grookey,Starter,
2,3,Rillaboom,Grass,,Gen 8 (SWSH),Thwackey,Evolve Grookey,
3,4,Scorbunny,Fire,,Gen 8 (SWSH),Rillaboom,Evolve Thwacket,
4,5,Raboot,Fire,,Gen 8 (SWSH),Scorbunny,Starter,
...,...,...,...,...,...,...,...,...
396,397,Dragapult,Dragon / Ghost,,Gen 8 (SWSH),Drakloak,Lake of Outrage,-
397,398,Zacian,Fairy (Fairy / Steel),,Gen 8 (SWSH),Dragapult,Evolve Drakloak,-
398,399,Zamazenta,Fighting (Fighting / Steel),,Gen 8 (SWSH),Zacian,Legendary,-
399,400,Eternatus,Poison / Dragon,,Gen 8 (SWSH),Zamazenta,Legendary,-


In [ ]:
url='https://www.polygon.com/pokemon-sword-shield-guide/2019/11/16/20968169/type-strength-weakness-super-effective-weakness-chart'

page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [ ]:
[len(T) for T in tr_elements[:12]]

In [ ]:
col = []
i = 0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d: "%s"'%(i,name))
    col.append((name,[]))

In [ ]:
col

In [ ]:
for j in range(1, len(tr_elements)):
    T = tr_elements[j]
    if len(T) != 3:
        break    
    i = 0
    
    for t in T.iterchildren():
        data=t.text_content()
        if i > 0:
            try:
                data = int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [ ]:
Dict={title:column for (title,column) in col}
df_battle=pd.DataFrame(Dict)

In [ ]:
df_battle = df_battle.drop(0)
df_battle = df_battle.reset_index(drop=True)

In [ ]:
df_battle

In [ ]:
pokemon_df